<a href="https://colab.research.google.com/github/as9786/NLP/blob/main/%EC%88%9C%ED%99%98%EC%8B%A0%EA%B2%BD%EB%A7%9D/%EC%88%9C%ED%99%98%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 순환신경망 구현 및 학습

In [1]:
import tensorflow as tf

# 초매개변수

In [2]:
EPOCHS = 10
NUM_WORDS = 10000

# 모형 정의

In [4]:
class MyModel(tf.keras.Model):

  def __init__(self):
    super(MyModel,self).__init__()
    self.emb = tf.keras.layers.Embedding(NUM_WORDS,16)
    self.rnn = tf.keras.layers.SimpleRNN(32) # LSTM, GRU
    self.dense = tf.keras.layers.Dense(2,activation='softmax')

  def call(self,x,training=None,mask=None):
    x = self.emb(x)
    x = self.rnn(x)
    return self.dense(x)

# Training and testing loop

In [5]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
  with tf.GradientTape() as tape:
    predictions = model(inputs,training=True)
    loss = loss_object(labels,predictions)

  gradients = tape.gradient(loss,model.trainable_variables) 

  optimizer.apply_gradients(zip(gradients,model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels,predictions)

@tf.function
def test_step(model, inputs, labels, loss_object, test_loss, test_accuracy):
  predictions = model(inputs,training=False)

  t_loss = loss_object(labels,predictions)
  test_loss(t_loss) 
  test_accuracy(labels,predictions)

# Dataset

In [6]:
imdb = tf.keras.datasets.imdb

(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words = NUM_WORDS)

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,value=0,padding='pre',maxlen=32) #post
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,value=0,padding='pre',maxlen=32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

17464789/17464789 [==============================] - 0s 0us/step


# 학습 환경 정의

In [7]:
model = MyModel()

In [8]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [9]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# 학습

In [10]:
for epoch in range(EPOCHS):
  for inputs, labels in train_ds:
    train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy)

  for test_inputs, test_labels in test_ds:
    test_step(model,test_inputs,test_labels,loss_object,test_loss,test_accuracy) 
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,train_loss.result(),train_accuracy.result()*100,test_loss.result(),
                        test_accuracy.result()*100))
  
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.531186580657959, Accuracy: 72.07599639892578, Test Loss: 0.4559605121612549, Test Accuracy: 78.35199737548828
Epoch 2, Loss: 0.3640669882297516, Accuracy: 84.23999786376953, Test Loss: 0.4865472912788391, Test Accuracy: 77.9959945678711
Epoch 3, Loss: 0.23300392925739288, Accuracy: 90.7760009765625, Test Loss: 0.5921752452850342, Test Accuracy: 76.52000427246094
Epoch 4, Loss: 0.10640458017587662, Accuracy: 96.3479995727539, Test Loss: 0.8308428525924683, Test Accuracy: 74.04800415039062
Epoch 5, Loss: 0.04441007971763611, Accuracy: 98.62800598144531, Test Loss: 1.0586357116699219, Test Accuracy: 75.05999755859375
Epoch 6, Loss: 0.018876325339078903, Accuracy: 99.52399444580078, Test Loss: 1.24251127243042, Test Accuracy: 74.45999908447266
Epoch 7, Loss: 0.015722103416919708, Accuracy: 99.47599792480469, Test Loss: 1.2953540086746216, Test Accuracy: 73.5199966430664
Epoch 8, Loss: 0.01858416758477688, Accuracy: 99.37200164794922, Test Loss: 1.389238715171814, Test Accu